In [1]:
import urllib
import os.path
import pathlib
import pandas as pd
import numpy as np
import functools
import itertools

from nameparser import HumanName
import humanize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 250)

In [2]:
def download_unless_isfile(url, file):
    if not os.path.isfile(file):
        urllib.request.urlretrieve(url, file)
    
download_unless_isfile("https://www.nyccfb.info/DataLibrary/Contribution_Key.csv", "Contribution_Key.csv")
download_unless_isfile("https://www.nyccfb.info/DataLibrary/2021_Contributions.csv", "2021_Contributions.csv")
download_unless_isfile("https://data.cityofnewyork.us/api/views/fmf3-knd8/rows.csv?accessType=DOWNLOAD", "eLobbyist_data.csv")


In [3]:
all_contributions = pd.read_csv("2021_Contributions.csv", header=0, dtype={ 'NAME': str, 'STRNO': str, 'STRNAME': str, 'APARTMENT': str , 'INTZIP': str, 'EXEMPTCD': str })

contributions = all_contributions[all_contributions.SCHEDULE.isin(["ABC", "IREF"])].copy()

In [4]:
def employer_format(s):
    if not isinstance(s, str):
        return s
    if s.lower() == 'not employed' or s.lower() == 'unemployed':
        return 'Not Employed'
    elif  s.lower() in ['self-employed', 'self employed', 'self', 'Self Employee']:
        return 'Self Employed'
    elif s.lower() == 'none':
        return 'None'
    elif s.lower() in ['homemaker', 'housewife']:
        return 'Homemaker'
    elif s.lower() in ['Citigroup', 'citi']:
        return 'Citigroup'
    elif 'paul weiss' in s.lower():
        return 'Paul Weiss'
    elif ('Skadden Arps' in s.lower() or s == 'Skadden'):
        return 'Skadden Arps'
    elif 'Rosen & Katz' in s or s == 'Wachtell Lipton':
        return 'Wachtell, Lipton, Rosen & Katz'
    elif 'A&E Real Estate' in s.lower():
        return 'A&E Real Estate'
    else:
        return s


contributions['EMPNAME'] = contributions['EMPNAME'].apply(employer_format)

In [5]:
def to_money(i):
     return "$" + humanize.intcomma(round(i))

def to_person_name(s):
    return str(HumanName(s))

def parse_name(s):
    names = []
    
    human_name = HumanName(s)
    
    names.append(f"{human_name.last}, {human_name.first}")
    
    if human_name.suffix:
        names.append(f"{human_name.last} {human_name.suffix}, {human_name.first}")
    
    if human_name.middle:
        names.append(f"{human_name.last}, {human_name.first} {human_name.middle}")
    
        if len(human_name.middle) > 1:
            names.append(f"{human_name.last}, {human_name.first} {human_name.middle[0]}")
    
    return names


def name_variations_from_file(file):
    variations = []
    
    with open(file, 'r') as f:
        for line in f:
            for variation in parse_name(line):
                variations.append(variation.upper())

    return variations
                
real_estate_names_from_littlesis = name_variations_from_file('./real_estate_names.txt')
worst_landlords = name_variations_from_file('./worst_landlords.txt')

In [6]:
lobbyists = pd.read_csv("eLobbyist_data.csv", header=0)
corporate_lobbyists = lobbyists[lobbyists["LOBBYIST_NAME"].str.contains(" (?:LLC|L\.P\.)", case=False, regex=True)]
all_lobbyist_orgs = lobbyists["LOBBYIST_NAME"].unique()
lobbyist_employees = set(itertools.chain(*map(lambda x: x.split(';'), lobbyists["LOBBYIST_EMPLOYEES"])))
corporate_lobbyist_employees = set(itertools.chain(*map(lambda x: x.split(';'), corporate_lobbyists["LOBBYIST_EMPLOYEES"])))

lobbyist_employees_variations = list(map(lambda s: s.upper(), itertools.chain(*map(parse_name, lobbyist_employees))))
corporate_lobbyist_employees_variations = list(map(lambda s: s.upper(), itertools.chain(*map(parse_name, corporate_lobbyist_employees))))

In [7]:
contributions['realestate_littlesis'] = contributions['NAME'].str.upper().isin(real_estate_names_from_littlesis)
contributions['worst_landlord'] = contributions['NAME'].str.upper().isin(worst_landlords)
contributions['works_in_realestate'] = contributions['OCCUPATION'].str.contains(r'(?i).*REAL ESTATE.*')
contributions['is_realestate'] = contributions['realestate_littlesis'] | contributions['worst_landlord'] | contributions['works_in_realestate']
contributions['is_lobbyist'] = (contributions['OCCUPATION'].str.upper().eq('LOBBYIST') |  contributions['NAME'].str.upper().isin(corporate_lobbyist_employees_variations))
contributions['is_corporation'] = contributions["C_CODE"].isin(["LLC", "CORP"])
contributions['flag'] = contributions['is_corporation'] | contributions['is_lobbyist'] | contributions['is_realestate']


In [8]:
major_candidates = [
    'Adams, Eric L',
    'Donovan, Shaun',
    'Garcia, Kathryn A',
    'McGuire, Raymond J',
    'Morales, Dianne',
    'Stringer, Scott M',
    'Wiley, Maya D',
    'Yang, Andrew'
]

mayor_candidates_names = contributions[contributions['OFFICECD'].isin(['1', '11'])]['RECIPNAME'].unique()
candidate_last_names = dict(zip(mayor_candidates_names, map(lambda s: HumanName(s).last, mayor_candidates_names)))
candidate_last_names.values()

dict_values(['Tirschwell', 'Wiley', 'Yang', 'Morales', 'McGuire', 'Stringer', 'Sliwa', 'Cullen', 'Garcia', 'Donovan', 'Menchaca', 'Adams', 'Mateo', 'Diaz', 'Pepitone', 'Taylor', 'AbdulMalik', 'Chang', 'Sutton', 'Rose', 'Prince', 'Wright', 'Guimaraes', 'Flores', 'Emilien', 'Foldenauer', 'Filipchenko', 'Laurel-Smith', 'Kavovit', 'Francis', 'Oremland', 'Nunez', 'Downs', 'Kaplan', 'Miles', 'Reaves', 'Coenen', 'Bunea', 'Pinto', 'Fitzgerald', 'Krietchman', 'Seidman', 'Seely'])

In [9]:
mayor_donations = contributions[contributions["RECIPNAME"].isin(major_candidates)].copy()

In [10]:
mayor_donations.NAME.value_counts()

Jordan, Tyron         63
Cole, Sarah           59
Perkins, D            58
Koo, Julia            56
Warren, Roxanne       56
                      ..
Bear, Amy              1
Johnson, Sylvester     1
Clarke, Harry          1
Marinaccio, Olivia     1
Tibbetts II, John      1
Name: NAME, Length: 74907, dtype: int64

In [11]:
mayor_donations['C_CODE'].unique()

array(['PCOMP', 'CORP', 'IND', 'CAN', 'EMPO', 'FAM', 'OTHR', 'PCOMC',
       'PCOMZ', 'LLC'], dtype=object)

In [12]:
def filter_flagged_contributions(df):
    return df[df['flag']]
    # return df[ (df['realestate_littlesis'] | df['worst_landlord'] |  df['works_in_realestate']  | df['does_lobbying'] | ( df['C_CODE'] == "LLC"))]


flagged_donations = filter_flagged_contributions(mayor_donations)
flagged_donations_major_candidates = flagged_donations[flagged_donations["RECIPNAME"].isin(major_candidates)]
flagged_donation_all_races = filter_flagged_contributions(contributions)


columns = ['ELECTION', 'RECIPID', 'RECIPNAME', 'COMMITTEE', 'NAME', 'AMNT', 'DATE', 'REFUNDDATE', 'CITY', 'STATE', 'ZIP', 'OCCUPATION', 'EMPNAME', 'C_CODE', 'realestate_littlesis', 'worst_landlord', 'works_in_realestate']

flagged_donations.sort_values(['RECIPNAME', 'DATE'])[columns]

,ELECTION,RECIPID,RECIPNAME,COMMITTEE,NAME,AMNT,DATE,REFUNDDATE,CITY,STATE,ZIP,OCCUPATION,EMPNAME,C_CODE,realestate_littlesis,worst_landlord,works_in_realestate
63374,2021,1545,"Adams, Eric L",L,"Cohen, Adir",5100.0,1/10/2019,NaN,Brooklyn,NY,11230,Real Estate,Renaissance Realty Group,IND,False,False,True
124630,2021,1545,"Adams, Eric L",L,"Gottlieb, Israel",1000.0,1/10/2019,NaN,Lakewood Township,NJ,8701,Real estate investor,Self Employed,IND,False,False,True
212793,2021,1545,"Adams, Eric L",L,"Matheus, Donald",500.0,1/10/2019,NaN,Brooklyn,NY,11238,Real estate,Albatross group,IND,False,False,True
321999,2021,1545,"Adams, Eric L",L,"Strulovitch, Baila",2000.0,1/10/2019,NaN,Brooklyn,NY,11205,Real Estate,Self Employee,IND,False,False,True
192053,2021,1545,"Adams, Eric L",L,"Levy, Danny",100.0,1/10/2020,NaN,Brooklyn,NY,11230,Real estate,Self Employed,IND,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346745,2021,2546,"Yang, Andrew",H,"Wang, Kevin",100.0,5/6/2021,NaN,Somerville,MA,2143,Software Engineer,Wellframe,IND,True,False,False
146638,2021,2546,"Yang, Andrew",H,"Holt, Giles",50.0,5/7/2021,NaN,New York,NY,10065,Real Estate,KBD Holdings,IND,False,False,True
188504,2021,2546,"Yang, Andrew",H,"Lee, Saehan",50.0,5/7/2021,NaN,Santa Monica,CA,90401,Manager Real Estate,Knowledge Universe,IND,False,False,True
103383,2021,2546,"Yang, Andrew",H,"Fishkind, Jason",2000.0,5/8/2021,NaN,Purchase,NY,10577,Real Estate,OOHH,IND,False,False,True


In [13]:
flagged_donations.sort_values(['RECIPNAME', 'DATE']).to_csv('flagged_donations.csv')
flagged_donation_all_races.sort_values(['RECIPNAME', 'DATE']).to_csv('flagged_donations_all_races.csv')

In [14]:
total_amount_recieved = mayor_donations[["RECIPNAME", "AMNT"]].groupby("RECIPNAME").sum().sort_values(by="AMNT", ascending=False)
total_amount_recieved["AMNT"]  = total_amount_recieved["AMNT"].apply(to_money)
total_amount_recieved = total_amount_recieved.rename_axis('Candidates').rename(columns={'AMNT': 'Amount'})
total_amount_recieved

,Amount
Candidates,
"McGuire, Raymond J","$10,093,818"
"Adams, Eric L","$4,739,061"
"Yang, Andrew","$3,600,747"
"Donovan, Shaun","$2,858,059"
"Stringer, Scott M","$2,839,137"
"Wiley, Maya D","$1,598,117"
"Garcia, Kathryn A","$1,242,811"
"Morales, Dianne","$860,549"


In [15]:
def largest_donors_per_candidate(donations, limit=15):
    """Dataframe of largest donors for each candidate"""
    agg = donations[["RECIPNAME", "AMNT", "NAME", "EMPNAME"]].groupby(["RECIPNAME", "NAME"]).agg({'AMNT': sum, 'EMPNAME': lambda s: s.unique() }) # .sort_values(by='AMNT', ascending=False) # .groupby(level=0, group_keys=False).head(20) # .reset_index() # ort_values(by='AMNT')  # .groupby(level=0, group_keys=False).apply(lambda x: x.sort_values(('AMNT', 'sum'), ascending=False))
        
    return agg['AMNT'].groupby('RECIPNAME', group_keys=False).apply(lambda x: x.sort_values(ascending=False).head(limit).apply(to_money)).to_frame()
    

In [16]:
largest_donors_per_candidate(mayor_donations, limit=30)

AMNT
RECIPNAME          NAME                                     
Adams, Eric L      Bensusan, Danny                   $12,100
                   Waisman, Shai                     $10,200
                   Guzel, Murat                      $10,100
                   Singh, Navdeep                    $10,000
                   Serova, Daria                     $10,000
                   Boutross, Diana                   $10,000
                   Bartlett, Gregory                  $7,600
                   Karandikar, Satish                 $7,100
                   Akcetin, Mehmet                    $7,000
                   David, Emanuel                     $6,600
                   Cerchione, Gregory                 $6,000
                   Razi, Sami Uddin                   $6,000
                   Gravante, Nicholas                 $5,750
                   Basusta, Yusuf                     $5,500
                   Daneshvar, Michelle                $5,350
                   Marino, John                       $5,250
                   Fuzaylova, Michael                 $5,100
                   Dalamal, Aroon                     $5,100
                   Barricelli, Louis                  $5,100
                   Hraki, O                           $5,100
                   Belt, Antonia                      $5,100
                   Gaudet, Anne M                     $5,100
                   Renaud, Stephen                    $5,100
                   georgiton, tony                    $5,100
                   Perez San Martin, Carlos           $5,100
                   glazer, jaf                        $5,100
                   Basaranlar, Tumay                  $5,100
                   Shamis, Alina                      $5,100
                   Perdomo, Jessica                   $5,100
                   Kosokhar, Ronen                    $5,100
Donovan, Shaun     Donovan, Shaun                    $19,100
                   Blumenstein, Ruth                 $15,300
                   Zabar, Eli                        $10,200
                   Fredericks, Devon                 $10,200
                   Bingham, Clara                    $10,200
                   Nolen, Eliot                      $10,200
                   Briganti, Stephen                 $10,200
                   Rockefeller, David                $10,200
                   Goodman, Sara                     $10,150
                   Niejelow, Alexander                $8,100
                   Hickenlooper, Robin                $8,000
                   Whang, John                        $6,100
                   Meyer, Audrey                      $5,600
                   Werner, Zachary                    $5,600
                   Donovan, Lucas                     $5,450
                   Wiese, Megan                       $5,450
                   Birch, Eugenie                     $5,350
                   Heilbut, Wilfred                   $5,200
                   Istel, Yves                        $5,200
                   Barber, David                      $5,150
                   Traber, Catherine                  $5,100
                   Wenig, Devin                       $5,100
                   Goldstein, Gordon                  $5,100
                   Begley, Michael                    $5,100
                   Sterne, James                      $5,100
                   Moore, John                        $5,100
                   Goldstein, Barbara                 $5,100
                   Goldrich, Robert                   $5,100
                   Trulson, Derek                     $5,100
                   McBride, Jonathan                  $5,100
Garcia, Kathryn A  Kramer, Mchael                     $2,500
                   Smith, Daniel                      $2,500
                   Lee, Michael                       $2,250
                   Sutton, Loree                      $2,166
                   Ascher, Kat

In [17]:
# Largest Real Estate Donors
largest_donors_per_candidate(flagged_donations_major_candidates, limit=30)

AMNT
RECIPNAME          NAME                              
Adams, Eric L      Boutross, Diana            $10,000
                   tamir, Elliot               $5,100
                   Demirjian, Alexis           $5,100
                   Borrok, Charles R.          $5,100
                   Cohen, Adir                 $5,100
                   Berman, Nathan              $5,100
                   Tawil, Eliot                $5,100
                   Verrone, Robert             $5,100
                   Schwartz, Yoel              $5,000
                   Shurka, Manny               $5,000
                   Landau, Solomon             $5,000
                   Corman, Jeffrey             $5,000
                   popkin, drew                $4,000
                   Neamonitakis, Michael       $4,000
                   Fisher, Avi                 $3,400
                   Vitucci Sr, Dominick        $3,400
                   Ward, Jesse                 $3,400
                   Gindi, jeffrey              $3,175
                   Anagnost, Dick              $3,000
                   Levkov, Viacheslav          $3,000
                   Silviano, John              $3,000
                   Iannucci, Robert            $2,500
                   Chassin, Jonathan           $2,500
                   Marchi, Dean                $2,500
                   Sciortino, Anthony          $2,500
                   Ewers, Sonia                $2,500
                   Basile, Rocco               $2,250
                   Okada, Chris                $2,250
                   Ghershenzon, Mordechai      $2,000
                   Hidary, Abraham             $2,000
Donovan, Shaun     Milstein, Constance J       $5,100
                   Moore, John                 $5,100
                   Zeiler, John                $5,100
                   Winn, Arthur                $5,100
                   Elbaum, Abigail Black       $5,100
                   Clark, Ric                  $5,100
                   Wilson, John                $5,100
                   Fish, John F                $5,100
                   Fisher, Winston             $5,100
                   Friedman, Eileen            $5,100
                   Trulson, Derek              $5,100
                   Brodsky, Daniel             $5,100
                   DeBode, Gary                $5,100
                   Bernstein, Joshua           $5,100
                   Greenberg, Mark             $5,100
                   Stacom, Darcy               $5,100
                   Silverstein, Larry          $5,100
                   Silvers, Nicholas           $5,100
                   KATZ, JEFFREY               $5,100
                   Bahar, Aaron                $5,100
                   olshan, andrea              $5,100
                   Sheller, Chris              $5,100
                   Burch, Melissa              $5,075
                   Lake, Robert                $3,500
                   Martin, Rob                 $3,000
                   kuriloff, josh              $2,500
                   Rubler, Neil                $2,500
                   Benenson, Lawrence          $2,500
                   Coatsworth, Anna            $2,500
                   Wiener, Alan                $2,250
Garcia, Kathryn A  Kerr, Zachary               $2,000
                   Koeppel, David J            $2,000
                   Kalikow, Peter              $2,000
                   MacArthur, Andrew           $2,000
                   Mosler, Wendy               $2,000
                   Field, Michael              $2,000
                   Durst, Helena               $2,000
                   Yatrakis, Demetrios         $1,500
                   Ezra, Brian                 $1,400
                   Sheehan, Timothy            $1,300
                   Thypin, Benjamin            $1,250
                   Wark, Jesse                 $1,050
                   Ghossein, Yara              $1,000

In [18]:
def cfb_link(name, only_mayor=False):
    # url = "https://www.nyccfb.info/FTMSearch/Candidates/Contributions?ec=2021&rt=can&ir={}"
    
    if only_mayor:
        url = "https://www.nyccfb.info/FTMSearch/Candidates/Contributions?ec=2021&rt=can&ofc=1%2C11&ir={}"
    else:
        url = "https://www.nyccfb.info/FTMSearch/Candidates/Contributions?ec=2021&rt=can&ir={}"
        

    return url.format(urllib.parse.quote_plus(name))


def all_top_donors_for(candidate):
        columns = ['RECIPNAME', 'DATE', 'AMNT', 'NAME', 'ZIP', 'OCCUPATION', 'EMPNAME']
        donations = mayor_donations[mayor_donations['RECIPNAME'] == candidate][columns]
        donations["name"] =  donations["NAME"].apply(to_person_name)
        donations.rename(columns={'NAME': 'cfb_name', 'OCCUPATION': 'cfb_occupation', 'EMPNAME': 'cfb_employer'}, inplace=True)
        donations['cfb_link'] = donations['cfb_name'].apply(cfb_link)
        return donations.sort_values('AMNT', ascending=False)

def real_estate_donors_for(candidate):
        columns = ['RECIPNAME', 'DATE', 'AMNT', 'NAME', 'ZIP', 'OCCUPATION', 'EMPNAME']
        donations = flagged_donations_major_candidates[flagged_donations_major_candidates['RECIPNAME'] == candidate][columns]
        donations["name"] =  donations["NAME"].apply(to_person_name)
        donations.rename(columns={'NAME': 'cfb_name', 'OCCUPATION': 'cfb_occupation', 'EMPNAME': 'cfb_employer'}, inplace=True)
        donations['cfb_link'] = donations['cfb_name'].apply(functools.partial(cfb_link, only_mayor=True))
        return donations.sort_values('AMNT', ascending=False)

In [19]:
real_estate_donors_for('Donovan, Shaun').head(10)

,RECIPNAME,DATE,AMNT,cfb_name,ZIP,cfb_occupation,cfb_employer,name,cfb_link
366523,"Donovan, Shaun",1/4/2021,5100.0,"Zeiler, John",10023,Real Estate,Hudson Housing Capital,John Zeiler,https://www.nyccfb.info/FTMSearch/Candidates/C...
28288,"Donovan, Shaun",6/23/2020,5100.0,"Bernstein, Joshua",20007,Real Estate Management,Bernstein Management Corp,Joshua Bernstein,https://www.nyccfb.info/FTMSearch/Candidates/C...
103359,"Donovan, Shaun",12/9/2020,5100.0,"Fisher, Winston",10023,Partner,Fisher Brothers,Winston Fisher,https://www.nyccfb.info/FTMSearch/Candidates/C...
109492,"Donovan, Shaun",3/19/2021,5100.0,"Friedman, Eileen",10019,Real Estate Consultant,RCS Real Estate,Eileen Friedman,https://www.nyccfb.info/FTMSearch/Candidates/C...
92776,"Donovan, Shaun",5/13/2021,5100.0,"Elbaum, Abigail Black",10028,Principal,"Ogden CAP Properties, LLC",Abigail Black Elbaum,https://www.nyccfb.info/FTMSearch/Candidates/C...
127122,"Donovan, Shaun",4/9/2021,5100.0,"Greenberg, Mark",6759,Real Estate,MGRE Co LLC,Mark Greenberg,https://www.nyccfb.info/FTMSearch/Candidates/C...
167795,"Donovan, Shaun",1/28/2021,5100.0,"KATZ, JEFFREY",10023,Real Estate Developer,Sherwood Equities Inc.,JEFFREY KATZ,https://www.nyccfb.info/FTMSearch/Candidates/C...
78375,"Donovan, Shaun",12/12/2020,5100.0,"DeBode, Gary",7901,Real Estate,Gallium Real Estate LLC,Gary DeBode,https://www.nyccfb.info/FTMSearch/Candidates/C...
61828,"Donovan, Shaun",2/21/2021,5100.0,"Clark, Ric",11201,Managing Partner,WatermanClark,Ric Clark,https://www.nyccfb.info/FTMSearch/Candidates/C...
225637,"Donovan, Shaun",4/12/2021,5100.0,"Milstein, Constance J",10014,Real Estate Owner/Principal,"Ogden CAP Properties, LLC",Constance J Milstein,https://www.nyccfb.info/FTMSearch/Candidates/C...


In [20]:
real_estate_donors_for('Adams, Eric L').head(10).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 325868 to 183492
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   RECIPNAME       10 non-null     object 
 1   DATE            10 non-null     object 
 2   AMNT            10 non-null     float64
 3   cfb_name        10 non-null     object 
 4   ZIP             10 non-null     object 
 5   cfb_occupation  10 non-null     object 
 6   cfb_employer    10 non-null     object 
 7   name            10 non-null     object 
 8   cfb_link        10 non-null     object 
dtypes: float64(1), object(8)
memory usage: 800.0+ bytes


In [21]:
len(flagged_donations[flagged_donations['RECIPNAME'] == 'Adams, Eric L']['NAME'].unique())

345

In [22]:
pathlib.Path("donations").mkdir(parents=True, exist_ok=True)

# Saves two CSV files for candidate
#  candidate_top_realestate_donors.csv
#  canddiate_top_donors.csv
# see real_estate_donors_for() and all_top_donors_for()
for cand in ['Adams, Eric L', 'Donovan, Shaun', 'Garcia, Kathryn A', 'McGuire, Raymond J', 'Stringer, Scott M', 'Yang, Andrew']:
    path = f"donations/{candidate_last_names.get(cand)}_top_realestate_donors.csv"
    real_estate_donors_for(cand).head(50).to_csv(path, index=False)
    
    path = f"donations/{candidate_last_names.get(cand)}_top_donors.csv"
    all_top_donors_for(cand).head(250).to_csv(path, index=False)
    


In [23]:
# Top Real Estate Donors
largest_donors_per_candidate(flagged_donations_major_candidates, limit=3)

AMNT
RECIPNAME          NAME                          
Adams, Eric L      Boutross, Diana        $10,000
                   tamir, Elliot           $5,100
                   Demirjian, Alexis       $5,100
Donovan, Shaun     Milstein, Constance J   $5,100
                   Moore, John             $5,100
                   Zeiler, John            $5,100
Garcia, Kathryn A  Kerr, Zachary           $2,000
                   Koeppel, David J        $2,000
                   Kalikow, Peter          $2,000
McGuire, Raymond J Colacino, Michael       $8,433
                   Goldberg, Jeffrey       $5,100
                   Trulson, Derek          $5,100
Morales, Dianne    Rosenthal, Richard        $500
                   Neveloff, Jay             $400
                   Gonzalez, Teresa          $400
Stringer, Scott M  Klein, John S          $10,100
                   Quinlan, Robert         $7,900
                   Fisher, Kenneth         $5,100
Wiley, Maya D      Shubert, William        $2,000
                   Khan, Fazeel            $2,000
                   Rubin, Alan             $1,750
Yang, Andrew       Shiah, Patricia         $2,250
                   Pollack, Allan          $2,000
                   Wei, Nelson             $2,000

In [24]:
def format_supported_candidates(names):
    out = set()
    
    for n in names:
        out.add(HumanName(n).last)
    
    return ", ".join(out)

In [25]:
def top_donors_for(df, head=20):
    top_donors = df.groupby(["NAME"]).agg({"AMNT": 'sum', 'RECIPNAME': lambda series: list(series.unique()) }).sort_values(by="AMNT", ascending=False).head(head).reset_index()
    top_donors.rename(columns={"NAME": "Donor Name", "AMNT": "Amount", "RECIPNAME": "Supported Candidates" }, inplace=True)
    top_donors['Donor Name'] = top_donors['Donor Name'].apply(to_person_name)
    top_donors["Amount"] = top_donors["Amount"].apply(lambda x: f"${humanize.intcomma(round(x))}" )
    top_donors["Supported Candidates"] = top_donors["Supported Candidates"].apply(format_supported_candidates)
    return top_donors


all_top_donors = top_donors_for(mayor_donations, head=50)
top_real_estate_donors = top_donors_for(flagged_donations, head=50)


In [26]:
top_real_estate_donors

,Donor Name,Amount,Supported Candidates
0,Winston Fisher,"$14,200","Stringer, Donovan, Adams, Yang"
1,Derek Trulson,"$12,200","McGuire, Donovan, Adams"
2,Daniel Brodsky,"$12,200","Stringer, Donovan, Yang"
3,John S Klein,"$10,100",Stringer
4,Diana Boutross,"$10,000",Adams
5,Kenneth Fisher,"$9,100","Stringer, Adams, Yang"
6,Michael Colacino,"$8,433",McGuire
7,Robert Quinlan,"$7,900",Stringer
8,John Wilson,"$7,610","McGuire, Donovan, Yang"
9,Andrew Brooks,"$7,100","McGuire, Adams"


In [27]:
all_top_donors

,Donor Name,Amount,Supported Candidates
0,Raymond J McGuire,"$1,000,000",McGuire
1,Shaun Donovan,"$19,100",Donovan
2,Anne Williams-Isom,"$17,300","McGuire, Wiley"
3,Ruth Blumenstein,"$15,300",Donovan
4,Allison Lutnick,"$15,300","McGuire, Donovan"
5,John Petry,"$14,200","Garcia, Yang, Adams, McGuire"
6,Winston Fisher,"$14,200","Stringer, Donovan, Adams, Yang"
7,James Tisch,"$14,200","Garcia, Adams, McGuire"
8,Steven Rattner,"$14,200","Stringer, McGuire, Donovan, Adams"
9,Judith Rubin,"$13,050","Morales, Donovan, Garcia, Stringer, McGuire, Yang"


In [28]:
# Top Employers Mayors
mayor_donations[["EMPNAME", "AMNT"]].groupby(["EMPNAME"]).agg(['sum', 'count']).sort_values(by=('AMNT', 'sum'), ascending=False).head(100)

AMNT       
                                       sum  count
EMPNAME                                          
Not Employed                    4957472.69  39042
Self Employed                   2982305.87  10165
Retired                          585065.19   1639
Paul Weiss                       206473.42    129
Citigroup                        159711.71    232
Homemaker                        118064.00    197
Skadden Arps                     101175.00    102
Self Employee                     85935.00    116
Wachtell, Lipton, Rosen & Katz    73825.00     57
Greenberg Traurig                 69275.00     35
None                              68661.46    114
Columbia University               54285.88    421
Google                            53283.68    406
City of New York                  50477.85    679
retired                           46738.31    117
Pershing Square Capital Manage    43200.00     11
NYU                               42834.59    361
Monroe College                    38430.00     37
Sullivan & Cromwell LLP           38430.00     37
Kirkland & Ellis LLP              37184.04     28
Lazard                            37000.00     10
Centerview Partners               34600.00      8
Skadden Arps Slate Meagher & F    33900.00     18
Cravath, Swaine & Moore LLP       31460.00     20
NYPD                              31330.81    277
Compass                           30121.21     78
Kirkland & Ellis                  29810.00     20
Student                           28760.10     92
NYC DOE                           26882.28    541
Fisher Brothers                   26455.00     11
CBRE                              26250.00     29
Morgan Stanley                    25115.21     63
Bank of America                   24682.42     94
PJT Partners                      24600.00     10
ViacomCBS                         24516.21     76
Mediaocean                        24161.00     46
Long Pond Capital                 23900.00      6
JLL                               23065.00     25
Willkie Farr & Gallagher LLP      22211.00     29
Standard Industries               21800.00      7
Cleary Gottlieb                   21600.00     15
Cleary Gottlieb Steen & Hamilt    21550.00     14
Greenberg Traurig, LLP            20700.00      5
Bloomberg                         20290.21     63
Fordham University                20153.42     66
DSNY                              19384.07    209
Joele Frank                       19001.75     13
CUNY                              18639.84    307
NYC                               18259.81    263
Shearman & Sterling LLP           18167.70     19
Simpson Thacher & Bartlett LLP    18005.00     18
Peloton                           17786.18     13
New York University               17746.31    241
Apple                             17413.05     78
King & Spalding LLP               17375.00     13
Sullivan & Cromwell               17320.00     21
Eminence Capital                  17300.00      4
Mastercard                        17120.00     37
King & Spalding                   17000.00     13
Third Point LLC                   16600.00     13
Douglas Elliman                   16575.00     35
Tashkent Supermarket              16000.00      7
Ehrenkranz Partners               15875.00      7
Fairstead                         15800.00      4
Estee Lauder Companies            15520.00      7
Corcoran                          15440.21     30
Facebook                          15402.25    124
Allen & Company LLC               15400.00      4
Greenberg Traurig LLP             15368.00      6
TG Holdings, LLC                  15300.00      3
Loews Corp                        15200.00      5
Rubenstein Communications         15150.00      3
Sessa Capital                     14950.00      7
Willett Advisors LLC              14200.00      4
The New School                    13978.51    124
Harvard Business School           13950.00      8
NYC Department of Education       13854.53    199
Newmark                           1385

In [29]:
# Top Employers Overall
contributions[["EMPNAME", "AMNT"]].groupby(["EMPNAME"]).agg(['sum', 'count']).sort_values(by=('AMNT', 'sum'), ascending=False).head(50)

AMNT       
                                       sum  count
EMPNAME                                          
Self Employed                   7368034.39  39735
Not Employed                    7170836.20  59056
Retired                         1240461.35   7439
Paul Weiss                       254804.42    301
Homemaker                        241004.30    677
Citigroup                        180437.71    391
City of New York                 154396.44   2104
NYC DOE                          120176.42   1958
retired                          113041.51    931
Skadden Arps                     111034.00    130
Columbia University              106625.45   1064
Monroe College                   105040.00    119
Google                            94978.37    827
None                              92445.38    460
Wachtell, Lipton, Rosen & Katz    88039.00     96
Self Employee                     85985.00    117
Greenberg Traurig                 81610.00    100
NYPD                              77357.30    877
NYU                               73842.59    746
RETIRED                           73733.08    972
NYC Council                       65695.98    779
NYCDOE                            65563.89   1001
NYC Department of Education       60814.10    763
Fisher Brothers                   58455.00     28
Student                           55025.31    578
Sullivan & Cromwell LLP           54050.00     85
New York City Council             54049.97    633
CUNY                              53131.13    900
Lazard                            48300.00     21
NYC                               46399.81    646
DOE                               44553.38    791
Compass                           44141.21    161
Morgan Stanley                    43635.21    166
Pershing Square Capital Manage    43200.00     11
CBRE                              42975.00     49
Standard Industries               41380.00     25
Kirkland & Ellis LLP              40814.04     44
Bank of America                   39704.63    233
Centerview Partners               39125.00     18
New York University               37358.40    536
Skadden Arps Slate Meagher & F    36137.00     31
ViacomCBS                         35409.21    167
Kirkland & Ellis                  33715.00     36
Cravath, Swaine & Moore LLP       33002.00     39
NYS Assembly                      32656.09    319
Fordham University                31715.26    216
Verizon                           31025.30    301
Broadway Stages                   28850.00     36
Willkie Farr & Gallagher LLP      28831.00     43
Bloomberg                         27640.21    150

In [30]:
sum_from_real_estate = flagged_donations_major_candidates[['RECIPNAME', 'AMNT']].groupby('RECIPNAME').sum()
sum_from_real_estate = sum_from_real_estate.sort_values(by='AMNT', ascending=False)
sum_from_real_estate['AMNT'] = sum_from_real_estate['AMNT'].apply(to_money)

sum_from_real_estate

,AMNT
RECIPNAME,
"McGuire, Raymond J","$318,276"
"Adams, Eric L","$299,143"
"Donovan, Shaun","$199,999"
"Stringer, Scott M","$161,316"
"Yang, Andrew","$121,662"
"Garcia, Kathryn A","$62,056"
"Wiley, Maya D","$17,146"
"Morales, Dianne","$4,661"


In [31]:
total_amount_recieved = mayor_donations[["RECIPNAME", "AMNT"]].groupby("RECIPNAME").sum().sort_values(by='RECIPNAME')
total_amount_recieved["Real Estate"] = mayor_donations[mayor_donations['is_realestate']][['RECIPNAME', 'AMNT']].groupby('RECIPNAME').sum().sort_values(by='RECIPNAME')
total_amount_recieved["Lobbyist"] = mayor_donations[mayor_donations['is_lobbyist']][['RECIPNAME', 'AMNT']].groupby('RECIPNAME').sum().sort_values(by='RECIPNAME')
# total_amount_recieved["Corporation"] = mayor_donations[mayor_donations['is_corporation']][['RECIPNAME', 'AMNT']].groupby('RECIPNAME').sum().sort_values(by='RECIPNAME')

total_amount_recieved = total_amount_recieved.reset_index().sort_values(by='AMNT', ascending=False)

total_amount_recieved['Name'] = total_amount_recieved['RECIPNAME'].apply(to_person_name)
total_amount_recieved["Total"]  = total_amount_recieved["AMNT"].apply(to_money)
total_amount_recieved["Real Estate"] = total_amount_recieved["Real Estate"].apply(to_money)
total_amount_recieved["Lobbyist"] = total_amount_recieved["Lobbyist"].apply(to_money)


total_amount_recieved[['Name', 'Total', 'Real Estate', 'Lobbyist']].style.hide_index()

Name,Total,Real Estate,Lobbyist
Raymond J McGuire,"$10,093,818","$285,375","$34,200"
Eric L Adams,"$4,739,061","$289,243","$16,700"
Andrew Yang,"$3,600,747","$112,529","$8,773"
Shaun Donovan,"$2,858,059","$189,402","$11,872"
Scott M Stringer,"$2,839,137","$151,556","$14,935"
Maya D Wiley,"$1,598,117","$13,477","$4,669"
Kathryn A Garcia,"$1,242,811","$56,531","$6,825"
Dianne Morales,"$860,549","$3,197","$1,464"


In [32]:
# Total of all donations to all major candidates
to_money(flagged_donations_major_candidates['AMNT'].sum())

'$1,184,259'

In [33]:
flagged_donations_major_candidates['AMNT'].sum() / mayor_donations['AMNT'].sum() * 100

4.2549817145556075

In [34]:
# Save CSV of top 100 employers
top_employers = mayor_donations[["EMPNAME", "AMNT"]].copy()
top_employers['EMPNAME'] = top_employers['EMPNAME'].apply(employer_format)

top_employers = top_employers.groupby('EMPNAME').sum().sort_values('AMNT', ascending=False).reset_index()[['EMPNAME', 'AMNT']].sort_values(by='AMNT', ascending=False)
top_employers['Employer Name'] = top_employers['EMPNAME']
top_employers['Amount'] = top_employers['AMNT'].apply(to_money)

top_employers[['Employer Name', 'Amount']].head(100).to_csv('top_100_employers.csv', index=False)

In [35]:
employer_agg = mayor_donations[["RECIPNAME", "EMPNAME", "AMNT"]].copy().groupby(["RECIPNAME", "EMPNAME"]).agg(['sum', 'count']).sort_values(by=('AMNT', 'sum'), ascending=False)
employer_agg.head(50)

AMNT       
                                                          sum  count
RECIPNAME          EMPNAME                                          
McGuire, Raymond J Not Employed                    1640676.68   2288
Yang, Andrew       Not Employed                    1002820.93  11955
McGuire, Raymond J Self Employed                    946095.00   1040
Donovan, Shaun     Not Employed                     861399.15   1824
Adams, Eric L      Self Employed                    846381.00   1451
Stringer, Scott M  Not Employed                     486720.92   4354
Wiley, Maya D      Not Employed                     449755.00  10824
Donovan, Shaun     Self Employed                    334462.84    580
McGuire, Raymond J Retired                          306499.97    273
Garcia, Kathryn A  Not Employed                     271229.44   2331
Stringer, Scott M  Self Employed                    248975.50   1186
Yang, Andrew       Self Employed                    221548.79   1754
Morales, Dianne    Not Employed                     208498.57   5166
McGuire, Raymond J Paul Weiss                       198390.42     99
Wiley, Maya D      Self Employed                    190220.25   2292
McGuire, Raymond J Citigroup                        151267.71    167
Garcia, Kathryn A  Self Employed                    126161.75    816
Adams, Eric L      Retired                          109607.00    393
McGuire, Raymond J Skadden Arps                      96935.00     85
Stringer, Scott M  Retired                           96573.88    642
Adams, Eric L      Self Employee                     85935.00    116
Morales, Dianne    Self Employed                     68460.74   1046
Stringer, Scott M  Homemaker                         67195.00    108
McGuire, Raymond J Greenberg Traurig                 64550.00     18
                   None                              64133.46     49
                   Wachtell, Lipton, Rosen & Katz    58600.00     29
Adams, Eric L      Homemaker                         45639.00     82
Wiley, Maya D      Retired                           44677.63    277
McGuire, Raymond J Pershing Square Capital Manage    39200.00      9
                   Lazard                            36600.00      9
Adams, Eric L      Not Employed                      36372.00    300
McGuire, Raymond J Sullivan & Cromwell LLP           35050.00     21
                   Skadden Arps Slate Meagher & F    33800.00     17
Stringer, Scott M  Monroe College                    32400.00     23
McGuire, Raymond J Cravath, Swaine & Moore LLP       31100.00     15
                   Centerview Partners               30600.00      6
                   Kirkland & Ellis LLP              28925.00     20
Adams, Eric L      NYPD                              28775.00    220
McGuire, Raymond J Kirkland & Ellis                  26600.00     11
                   retired                           25868.15     19
                   PJT Partners                      24600.00     10
Donovan, Shaun     Mediaocean                        24161.00     46
McGuire, Raymond J Long Pond Capital                 23900.00      6
Donovan, Shaun     Retired                           22180.00     10
Garcia, Kathryn A  City of New York                  21665.65    246
McGuire, Raymond J Cleary Gottlieb Steen & Hamilt    21450.00     13
                   Compass                           21150.21     23
                   Greenberg Traurig, LLP            20700.00      5
                   Standard Industries               19800.00      6
                   Willkie Farr & Gallagher LLP      19400.00     14

In [36]:
def employer_donations(name):    
    return mayor_donations[mayor_donations["EMPNAME"].str.lower() == name.lower()]

def employer_donations_contains(name):    
    return mayor_donations[mayor_donations["EMPNAME"].str.lower().contains(name.lower)]

def candidates_supported(employer_name):
    recipients = employer_donations(employer_name)[["RECIPNAME", "AMNT"]].groupby('RECIPNAME').sum().reset_index().sort_values('AMNT', ascending=False).to_dict('records')
    return list(map(lambda x: f"{to_person_name(x['RECIPNAME'])} {to_money(x['AMNT'])}", recipients))

def employer_donations_summary(name, include_donors=False):
    d = { 'name': name }
    d['sum'] = round(employer_donations(name)['AMNT'].sum())
    d['recipients'] = candidates_supported(name)
    
    donors = list(map(to_person_name, employer_donations(name)["NAME"].unique()))
    
    d['donor_count'] = len(donors)
    d['donor_avg'] = to_money(d['sum'] / d['donor_count'])
    
    if include_donors:
        d['donors'] = donors
    
    return d
    

In [37]:
employer_donations_summary("NETFLIX")

{'name': 'NETFLIX',
 'sum': 12411,
 'recipients': ['Raymond J McGuire $10,350',
  'Shaun Donovan $1,275',
  'Andrew Yang $331',
  'Scott M Stringer $300',
  'Dianne Morales $100',
  'Maya D Wiley $55'],
 'donor_count': 17,
 'donor_avg': '$730'}

In [38]:
employer_donations_summary("Kirkland & Ellis", include_donors=True)

{'name': 'Kirkland & Ellis',
 'sum': 29810,
 'recipients': ['Raymond J McGuire $26,600',
  'Scott M Stringer $2,400',
  'Shaun Donovan $400',
  'Andrew Yang $300',
  'Kathryn A Garcia $100',
  'Maya D Wiley $10'],
 'donor_count': 18,
 'donor_avg': '$1,656',
 'donors': ['reginald brown',
  'Lauren Friedman',
  'Daniel Fuglestad',
  'Jacqueline Haberfeld',
  'Jonathan Henes',
  'Jia Li Huang',
  'Melissa Hutson',
  'Jason Kanner',
  'Michael Kim',
  'Peter Leung',
  'Song Lin',
  'Ramiro Ocasio',
  'Jen Perkins',
  'Edward Sassower',
  'Monica Shilling',
  'Eric Wedel',
  'Erica Williams',
  'Ali Zaidi']}

In [39]:
employer_donations_summary("Paul Weiss")

{'name': 'Paul Weiss',
 'sum': 206473,
 'recipients': ['Raymond J McGuire $198,390',
  'Andrew Yang $4,325',
  'Maya D Wiley $2,170',
  'Scott M Stringer $600',
  'Shaun Donovan $350',
  'Kathryn A Garcia $315',
  'Dianne Morales $273',
  'Eric L Adams $50'],
 'donor_count': 107,
 'donor_avg': '$1,930'}

In [40]:
employer_donations_summary("Google")

{'name': 'Google',
 'sum': 53484,
 'recipients': ['Andrew Yang $18,252',
  'Raymond J McGuire $12,065',
  'Kathryn A Garcia $7,810',
  'Dianne Morales $6,012',
  'Shaun Donovan $3,335',
  'Maya D Wiley $3,004',
  'Scott M Stringer $2,605',
  'Eric L Adams $400'],
 'donor_count': 240,
 'donor_avg': '$223'}

In [41]:
employer_donations_summary("Apple")

{'name': 'Apple',
 'sum': 17463,
 'recipients': ['Shaun Donovan $5,875',
  'Raymond J McGuire $5,350',
  'Maya D Wiley $2,405',
  'Andrew Yang $2,303',
  'Dianne Morales $1,354',
  'Eric L Adams $100',
  'Kathryn A Garcia $50',
  'Scott M Stringer $25'],
 'donor_count': 57,
 'donor_avg': '$306'}

In [42]:
employer_donations_summary("A&E Real Estate")

{'name': 'A&E Real Estate',
 'sum': 15100,
 'recipients': ['Raymond J McGuire $12,700',
  'Eric L Adams $2,000',
  'Kathryn A Garcia $400'],
 'donor_count': 5,
 'donor_avg': '$3,020'}

In [43]:
employer_donations_summary("Long Pond Capital")

{'name': 'Long Pond Capital',
 'sum': 23900,
 'recipients': ['Raymond J McGuire $23,900'],
 'donor_count': 6,
 'donor_avg': '$3,983'}

In [44]:
all_lobbyist_contributions = contributions[contributions['NAME'].str.upper().isin(lobbyist_employees_variations) | contributions['OCCUPATION'].str.upper().eq('LOBBYIST')]
lobbyist_contributions = mayor_donations[mayor_donations['NAME'].str.upper().isin(lobbyist_employees_variations) | mayor_donations['OCCUPATION'].str.upper().eq('LOBBYIST')]
corporate_lobbyist_contributions = mayor_donations[mayor_donations['NAME'].str.upper().isin(corporate_lobbyist_employees_variations) | mayor_donations['OCCUPATION'].str.upper().eq('LOBBYIST')]

In [45]:
lobbyist_contributions

,ELECTION,OFFICECD,RECIPID,CANCLASS,RECIPNAME,COMMITTEE,FILING,SCHEDULE,PAGENO,SEQUENCENO,REFNO,DATE,REFUNDDATE,NAME,C_CODE,STRNO,STRNAME,APARTMENT,BOROUGHCD,CITY,STATE,ZIP,OCCUPATION,EMPNAME,EMPSTRNO,EMPSTRNAME,EMPCITY,EMPSTATE,AMNT,MATCHAMNT,PREVAMNT,PAY_METHOD,INTERMNO,INTERMNAME,INTSTRNO,INTSTRNM,INTAPTNO,INTCITY,INTST,INTZIP,INTEMPNAME,INTEMPSTNO,INTEMPSTNM,INTEMPCITY,INTEMPST,INTOCCUPA,PURPOSECD,EXEMPTCD,ADJTYPECD,RR_IND,SEG_IND,INT_C_CODE,realestate_littlesis,worst_landlord,works_in_realestate,is_realestate,is_lobbyist,is_corporation,flag
2073,2021,1,2303,P,"Donovan, Shaun",H,6,ABC,NaN,NaN,R0003122,9/13/2020,NaN,"Adams, Amelia",IND,NaN,NaN,NaN,M,New York,NY,10004,Consultant,Adams Advisors LLC,One,Battery Park Plaza,New York,NY,50.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,True,False,True
2074,2021,1,2303,P,"Donovan, Shaun",H,7,ABC,NaN,NaN,R0014467,3/6/2021,NaN,"Adams, Amelia",IND,NaN,NaN,NaN,M,New York,NY,10004,Consultant,Adams Advisors LLC,One,Battery Park Plaza,New York,NY,50.0,0,50.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,True,False,True
2075,2021,1,2303,P,"Donovan, Shaun",H,7,ABC,NaN,NaN,R0014962,3/9/2021,NaN,"Adams, Amelia",IND,NaN,NaN,NaN,M,New York,NY,10004,Consultant,Adams Advisors LLC,One,Battery Park Plaza,New York,NY,100.0,0,100.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,True,False,True
2147,2021,11,2242,P,"Morales, Dianne",H,7,ABC,NaN,NaN,R0028213,3/10/2021,NaN,"Adams, Elizabeth",IND,NaN,NaN,NaN,K,Brooklyn,NY,11222,Legislative director,Nyc council,410,Atlantic avenue,Brooklyn,NY,40.0,40,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
2148,2021,11,2242,P,"Morales, Dianne",H,8,ABC,NaN,NaN,R0046578,5/17/2021,NaN,"Adams, Elizabeth",IND,NaN,NaN,NaN,K,Brooklyn,NY,11222,Legislative director,Nyc council,410,Atlantic avenue,Brooklyn,NY,21.0,21,40.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364163,2021,11,2393,P,"Wiley, Maya D",H,8,ABC,NaN,NaN,R0070262,5/17/2021,NaN,"Youdelman, Sondra",IND,NaN,NaN,NaN,K,Brooklyn,NY,11238,Campaigns Director,People's Action,32,Clifton Place,Brooklyn,NY,50.0,50,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
364492,2021,11,2393,P,"Wiley, Maya D",H,8,ABC,NaN,NaN,R0051865,4/16/2021,NaN,"Young, Patrick",IND,NaN,NaN,NaN,Q,East Elmhurst,NY,11369,Pastor,First Baptist Church,100-10,Astoria Blvd,East Elmhurst,NY,100.0,100,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
365085,2021,11,2242,P,"Morales, Dianne",H,8,ABC,NaN,NaN,R0047765,5/17/2021,NaN,"Yuan, Yaya",IND,NaN,NaN,NaN,M,New York,NY,10034,Program Director,Apex for Youth,120,Walker St.,New York,NY,50.0,50,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
365171,2021,11,1545,P,"Adams, Eric L",L,1,ABC,NaN,NaN,R0005372,7/12/2018,NaN,"Yuille, Antonia",IND,NaN,NaN,NaN,K,Brooklyn,NY,11213,Director Corporate Affairs,Con Edison,30,Flatbush Ave,Brooklyn,NY,50.0,0,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False


In [46]:
all_lobbyist_contributions

,ELECTION,OFFICECD,RECIPID,CANCLASS,RECIPNAME,COMMITTEE,FILING,SCHEDULE,PAGENO,SEQUENCENO,REFNO,DATE,REFUNDDATE,NAME,C_CODE,STRNO,STRNAME,APARTMENT,BOROUGHCD,CITY,STATE,ZIP,OCCUPATION,EMPNAME,EMPSTRNO,EMPSTRNAME,EMPCITY,EMPSTATE,AMNT,MATCHAMNT,PREVAMNT,PAY_METHOD,INTERMNO,INTERMNAME,INTSTRNO,INTSTRNM,INTAPTNO,INTCITY,INTST,INTZIP,INTEMPNAME,INTEMPSTNO,INTEMPSTNM,INTEMPCITY,INTEMPST,INTOCCUPA,PURPOSECD,EXEMPTCD,ADJTYPECD,RR_IND,SEG_IND,INT_C_CODE,realestate_littlesis,worst_landlord,works_in_realestate,is_realestate,is_lobbyist,is_corporation,flag
185,2021,55,2190,P,"Rein, Briget",H,1,ABC,NaN,NaN,R0000043,6/11/2018,NaN,"Aaronson, Melvyn",IND,NaN,NaN,NaN,M,New York,NY,10016,Retired,Retired,NaN,NaN,NaN,NaN,100.0,0,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
319,2021,11,1694,P,"Menchaca, Carlos",K,6,ABC,NaN,NaN,R0006214,12/3/2020,NaN,"Abbate, Mary",IND,NaN,NaN,NaN,Q,Fresh Meadows,NY,11366,Social Worker Adminstrator,Queens Community House,NaN,NaN,NaN,NaN,10.0,10,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
471,2021,55,2543,P,"Swinton, Victor",H,6,ABC,NaN,NaN,R0000027,1/5/2021,NaN,"Abdul-Jabbar, Mubarak",IND,NaN,NaN,NaN,X,NY,NY,10475,Retired,NaN,NaN,NaN,NaN,NaN,500.0,175,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
1323,2021,44,1301,P,"Kallos, Benjamin J",L,8,ABC,NaN,NaN,R0017414,4/6/2021,NaN,"Abreu, Ysabel",IND,NaN,NaN,NaN,M,New York,NY,10029,Community Planning Board Coord,Manhattan Borough Presidents O,311,East 104th St,New York,NY,50.0,50,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
1324,2021,44,1301,P,"Kallos, Benjamin J",L,8,ABC,NaN,NaN,R0018137,5/6/2021,NaN,"Abreu, Ysabel",IND,NaN,NaN,NaN,M,New York,NY,10029,Community Planning Board Coord,Manhattan Borough Presidents O,311,East 104th St,New York,NY,10.0,10,50.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366302,2021,6,264,NP,"Katz, Melinda",R,1,ABC,NaN,NaN,R0010066,2/2/2018,NaN,"Zausner, Daniel",IND,NaN,NaN,NaN,Z,Tenafly,NJ,7670,COO,USTA National Tennis Center,1,Flushing Meadows Corona Park,Corona,NY,400.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
366303,2021,55,380,P,"Moya, Francisco P",K,5,ABC,NaN,NaN,R0003206,2/7/2020,NaN,"zausner, daniel",IND,NaN,NaN,NaN,Z,Tenafly,NJ,7670,COO,usta national tennis center in,one,Flushing Meadows Corona Park,flushing,NY,250.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
367924,2021,55,2557,P,"Chen, Yi",H,7,ABC,NaN,NaN,R0000806,1/27/2021,NaN,"Zheng, Zhi Q",IND,NaN,NaN,NaN,Z,Mars,PA,16046,Unemployed,NaN,NaN,NaN,NaN,NaN,200.0,0,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False
368813,2021,33,2474,P,"Iscol, Zachary",H,6,ABC,NaN,NaN,R0002448,12/9/2020,NaN,"Zipkin, Rebecca",IND,NaN,NaN,NaN,Z,Westport,CT,6880,Attorney,World WE,150,W 28th St,New York,NY,125.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False,False,False,False


In [47]:
num_unique_names = len(lobbyist_contributions['NAME'].unique())
total_amount = to_money(lobbyist_contributions['AMNT'].sum())

"{} lobbyists have given {} to mayoral candidates".format(num_unique_names, total_amount)

'734 lobbyists have given $319,369 to mayoral candidates'

In [48]:
num_unique_names = len(corporate_lobbyist_contributions['NAME'].unique())
total_amount = to_money(corporate_lobbyist_contributions['AMNT'].sum())

"{} corporate lobbyists have given {} to mayoral candidates".format(num_unique_names, total_amount)

'147 corporate lobbyists have given $99,439 to mayoral candidates'

In [49]:
largest_donors_per_candidate(lobbyist_contributions, limit=5)

AMNT
RECIPNAME          NAME                        
Adams, Eric L      Gray, William         $5,000
                   Ng, David             $2,500
                   Wachtel, Madeline     $2,000
                   McCormick, Maggie     $2,000
                   Mack, William         $2,000
Donovan, Shaun     Wilson, John          $5,100
                   Miller, Elizabeth     $2,000
                   Merrill, Thomas       $2,000
                   Smith, Michael        $1,000
                   Greenberger, Sharon   $1,000
Garcia, Kathryn A  Stern, Andrew         $1,150
                   Murphy, Michael       $1,100
                   Mansky, Adam          $1,000
                   Bystryn, Marcia       $1,000
                   Johnson, Jennifer       $750
McGuire, Raymond J Ford, Mary           $10,200
                   Klein, Michael        $5,100
                   Goldberg, Jeffrey     $5,100
                   Boxer, Leonard        $5,100
                   Williams, Steven      $5,100
Morales, Dianne    Little, Keith         $2,350
                   Arroyo, Maritza         $675
                   Davis, Gordon           $650
                   Watler, Christopher     $500
                   Jackson, Michelle       $450
Stringer, Scott M  Fisher, Kenneth       $5,100
                   Kelly, Mary           $3,000
                   Rock, Ron             $3,000
                   Levine, Randy         $3,000
                   Lespinasse, Patrick   $2,000
Wiley, Maya D      Rubin, Alan           $1,750
                   Johnson, Michael      $1,000
                   Baxter, Ryan            $925
                   Hinds, Janella          $750
                   Hirsh, Alison           $553
Yang, Andrew       Lee, Eric             $2,050
                   Miller, Richard       $2,000
                   Williams, Steven      $2,000
                   Silverman, Eric       $2,000
                   Fisher, Kenneth       $2,000

In [50]:
largest_donors_per_candidate(corporate_lobbyist_contributions, limit=5)

AMNT
RECIPNAME          NAME                       
Adams, Eric L      Fisher, Kenneth      $2,000
                   McCormick, Maggie    $2,000
                   Lombino, David       $1,000
                   Whelan, Rory         $1,000
                   Wallace, William     $1,000
Donovan, Shaun     Wilson, John         $5,100
                   Merrill, Thomas      $2,000
                   Gilmartin, MaryAnne    $400
                   Frommer, Ross          $400
                   Ricks, Marc            $400
Garcia, Kathryn A  Gallagher, john        $500
                   Johnson, Michael       $500
                   Anfang, Alix           $425
                   Daly, Samara           $425
                   kimball, andrew        $400
McGuire, Raymond J Klein, Michael       $5,100
                   Williams, Steven     $5,100
                   Goldberg, Jeffrey    $5,100
                   Roth, Steven         $5,100
                   Wilson, John         $2,500
Morales, Dianne    Gonzalez, Teresa       $400
                   Parvizi, Nazli         $300
                   Pitta, Vincent         $150
                   Smith, Robert          $135
                   Farrell, William        $80
Stringer, Scott M  Fisher, Kenneth      $5,100
                   Sunshine, Kenneth    $2,000
                   Lespinasse, Patrick  $2,000
                   Sorensen, Mark         $500
                   Dames, Cynthia         $500
Wiley, Maya D      Rubin, Alan          $1,750
                   Johnson, Michael     $1,000
                   Fontas, George         $250
                   MARTINEZ, RAMON        $250
                   Goldberg, Jeffrey      $222
Yang, Andrew       Cohen, Michael       $2,000
                   Williams, Steven     $2,000
                   Fisher, Kenneth      $2,000
                   Gerstman, Bradley    $1,000
                   Jones, Robert          $400

In [51]:
def group_by_recipient_and_amount(df):
    group_df = df[["RECIPNAME", "AMNT"]].groupby('RECIPNAME').sum().sort_values(by="AMNT", ascending=False)
    group_df["AMNT"] = group_df["AMNT"].apply(to_money)
    return group_df.rename_axis('Candidates').rename(columns={'AMNT': 'Amount'})


In [52]:
# Lobbyist Donations to Mayoral Candidates
group_by_recipient_and_amount(lobbyist_contributions)

,Amount
Candidates,
"McGuire, Raymond J","$90,405"
"Adams, Eric L","$51,725"
"Stringer, Scott M","$50,155"
"Garcia, Kathryn A","$32,747"
"Donovan, Shaun","$28,363"
"Yang, Andrew","$26,559"
"Morales, Dianne","$19,852"
"Wiley, Maya D","$19,562"


In [53]:
# Corporate Lobbyist Donations to Mayoral Campaigns
group_by_recipient_and_amount(corporate_lobbyist_contributions)

,Amount
Candidates,
"McGuire, Raymond J","$34,200"
"Adams, Eric L","$16,700"
"Stringer, Scott M","$14,935"
"Donovan, Shaun","$11,872"
"Yang, Andrew","$8,773"
"Garcia, Kathryn A","$6,825"
"Wiley, Maya D","$4,669"
"Morales, Dianne","$1,464"


In [54]:
lobbyist_contributions.sort_values(['RECIPNAME', 'DATE']).to_csv('flagged_lobbyist_donations.csv')

In [55]:
group_by_recipient_and_amount(contributions)

,Amount
Candidates,
"McGuire, Raymond J","$10,093,818"
"Adams, Eric L","$4,739,061"
"Yang, Andrew","$3,600,747"
"Donovan, Shaun","$2,858,059"
"Stringer, Scott M","$2,839,137"
...,...
"Johnson, Vincent C",$55
"Marin, Danny",$40
"Estrella, Miguel",$25


In [56]:
group_by_recipient_and_amount(all_lobbyist_contributions).head(50)

,Amount
Candidates,
"McGuire, Raymond J","$90,405"
"Adams, Eric L","$51,725"
"Stringer, Scott M","$50,155"
"Garcia, Kathryn A","$32,747"
"Lander, Brad","$29,490"
"Levine, Mark","$29,303"
"Benjamin, Brian A","$29,033"
"Donovan, Shaun","$28,363"
"Yang, Andrew","$26,559"


In [57]:
to_money(all_lobbyist_contributions['AMNT'].sum())

'$946,716'